In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.width', 250)
pd.set_option('display.max_colwidth', 120)

### Reading CSV

In [3]:
data_dir = "data"
file_name = "Izskatitas_C1_Kat3_2017_lietas_07.csv"
data_source_path = os.path.join(data_dir, file_name)

In [4]:
df = pd.read_csv(data_source_path, parse_dates=["Ienākšanas datums", "Izskatīšanas datums"])

In [5]:
def make_float(x):
    try:
        return float(x)
    except ValueError:
        return np.nan
    
df["Prasījuma apmērs"] = df.apply(lambda item: make_float(item["Prasījuma apmērs"]), axis=1)

df["Būtība"] = df["Būtība"].astype(str)
df["Tiesas ID (courtID)"] = df["Tiesas ID (courtID)"].astype(str)

### Overall stats

In [6]:
df.shape

(16455, 18)

In [7]:
df.head()

,Lietas ID (caseID),Tiesas ID (courtID),Tiesa,Dzimšanas datums,Ienākšanas kārtība,Ienākšanas datums,Izskatīšanas datums,Būtība,Prasījuma apmērs,atb_fiz,atb_jur,pras_fiz,pras_jur,Nozīmētās sēdes,Notikušās sēdes,naudaS_sods,dzivo_arzemes,atbr_no_nodevas
0,213477,40,Valkas rajona tiesa,60159,Pirmo reizi,2003-06-05,2017-01-23,parāda piedziņa,0.00,0,1,0,1,3,0,0,1,0
1,448971,32,Rīgas pilsētas Vidzemes priekšpilsētas tiesa,50767,Pirmo reizi,2005-11-29,2017-04-06,zaudējumu piedziņa,0.00,1,0,1,0,9,1,0,0,1
2,477470,279,Rīgas pilsētas Pārdaugavas tiesa,10551,Sakarā ar tiesu apvienošanu,2017-01-31,2017-10-02,parāda piedziņa,21414.34,2,0,1,0,5,3,0,0,0
3,575613,27,Liepājas tiesa,30463,Pirmo reizi,2007-06-04,2017-10-26,zaudējumu piedziņa,916.25,6,0,2,0,4,0,0,0,0
4,580131,32,Rīgas pilsētas Vidzemes priekšpilsētas tiesa,250174,Pēc piekritības no citas tiesas,2015-10-07,2017-11-13,parāda piedziņa,1588192.95,4,0,1,0,8,1,0,0,0


In [8]:
df.dtypes

Lietas ID (caseID)              int64
Tiesas ID (courtID)            object
Tiesa                          object
Dzimšanas datums                int64
Ienākšanas kārtība             object
Ienākšanas datums      datetime64[ns]
Izskatīšanas datums    datetime64[ns]
Būtība                         object
Prasījuma apmērs              float64
atb_fiz                         int64
atb_jur                         int64
pras_fiz                        int64
pras_jur                        int64
Nozīmētās sēdes                 int64
Notikušās sēdes                 int64
naudaS_sods                     int64
dzivo_arzemes                   int64
atbr_no_nodevas                 int64
dtype: object

In [9]:
df.isnull().sum()

Lietas ID (caseID)     0
Tiesas ID (courtID)    0
Tiesa                  0
Dzimšanas datums       0
Ienākšanas kārtība     0
Ienākšanas datums      0
Izskatīšanas datums    0
Būtība                 0
Prasījuma apmērs       9
atb_fiz                0
atb_jur                0
pras_fiz               0
pras_jur               0
Nozīmētās sēdes        0
Notikušās sēdes        0
naudaS_sods            0
dzivo_arzemes          0
atbr_no_nodevas        0
dtype: int64

In [10]:
def categorical_stats(df):
    values_df = df.select_dtypes(include="object").nunique().to_frame(name="unique")
    values_df["values"] = values_df.apply(lambda item: df[item.name].unique().tolist(), axis=1)
    return values_df

In [11]:
categorical_stats(df)

,unique,values
Tiesas ID (courtID),30,"[40, 32, 279, 27, 25, 30, 26, 4, 5, 29, 34, 273, 19, 39, 23, 6, 16, 28, 1, 38, 10, 18, 2, 33, 8, 22, 7, 31, 37, 284]"
Tiesa,30,"[Valkas rajona tiesa, Rīgas pilsētas Vidzemes priekšpilsētas tiesa, Rīgas pilsētas Pārdaugavas tiesa, Liepājas tiesa..."
Ienākšanas kārtība,6,"[Pirmo reizi, Sakarā ar tiesu apvienošanu, Pēc piekritības no citas tiesas, Pēc lēmuma atcelšanas, CPL 32.1 panta kā..."
Būtība,1254,"[parāda piedziņa, zaudējumu piedziņa, par rokas naudas piedziņu, par parāda piedziņu, parāda piedziņa, prasības nodr..."


### Cleanup

In [12]:
df.dropna(subset=["Prasījuma apmērs"], inplace=True)
df.drop(labels=["Tiesa"], axis=1, inplace=True)

### Transformations

In [13]:
df["Būtība"] = df.apply(lambda item: " ".join(item["Būtība"].split()), axis=1)

In [14]:
categorical_stats(df)

,unique,values
Tiesas ID (courtID),30,"[40, 32, 279, 27, 25, 30, 26, 4, 5, 29, 34, 273, 19, 39, 23, 6, 16, 28, 1, 38, 10, 18, 2, 33, 8, 22, 7, 31, 37, 284]"
Ienākšanas kārtība,6,"[Pirmo reizi, Sakarā ar tiesu apvienošanu, Pēc piekritības no citas tiesas, Pēc lēmuma atcelšanas, CPL 32.1 panta kā..."
Būtība,1158,"[parāda piedziņa, zaudējumu piedziņa, par rokas naudas piedziņu, par parāda piedziņu, parāda piedziņa, prasības nodr..."
